In [1]:
# Install necessary packages
!pip install ultralytics wandb
!pip install -U ipywidgets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 733.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 793.0/793.0 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.4/214.4 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 24.1 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.6
    Uninstalling widgetsnbextension-3.6.6:
      Successfully uninstalled widgetsnbextension-3.6.6
  Attempting uninstall: jupyterlab-widgets
    Found existing installation: jupyterlab-widgets 3.0.9
    Uninstalling jupyterlab-widgets-3.0.9:
      Successfully uninstalled jupyterlab-widgets-3.0.9
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1


In [2]:
import yaml

# Define the path to the existing data.yaml file
yaml_path = '/kaggle/input/yolov8-dataset/Dataset/data.yaml'

# Read the content of the data.yaml file
with open(yaml_path, 'r') as file:
    data_config = yaml.safe_load(file)

# Modify the paths in the data.yaml content
data_config['path'] = '/kaggle/input/yolov8-dataset/Dataset'
data_config['train'] = '/kaggle/input/yolov8-dataset/Dataset/train/images'
data_config['val'] = '/kaggle/input/yolov8-dataset/Dataset/val/images'

# Save the modified content to the working directory
modified_yaml_path = '/kaggle/working/data.yaml'
with open(modified_yaml_path, 'w') as file:
    yaml.safe_dump(data_config, file)

print(f"Modified data.yaml file saved at: {modified_yaml_path}")


Modified data.yaml file saved at: /kaggle/working/data.yaml


In [3]:
# Import libraries
import wandb
from ultralytics import YOLO
import os

# Check GPU availability
!nvidia-smi

# Set environment variables for Weights & Biases
os.environ["WANDB_PROJECT"] = "YOLOv8NanoTrafficV2"
os.environ["WANDB_LOG_MODEL"] = "checkpoint"

# Login to wandb
wandb.login(key='c09731de5241aec4339bdabcb192b5bacc473a39')

# Initialize the model
model = YOLO('yolov8n.yaml')

# Define training parameters
kwargs = {
    'name': 'First',  # Change this to your run name
    'project': 'Traffic_Analysis',  # Change this to your project name
    'epochs': 1,  # Reduce the number of epochs for quicker training
    'save_period': 1,  # Save model checkpoint every 5 epochs
    'device': [0, 1],  # Specify GPUs to use
    'verbose': True,  # Enable verbose output
    'cos_lr': True,  # Use cosine learning rate schedule
    'lr0': 0.002,  # Initial learning rate
    'cls': 0.5,  # Classification loss weight
    'dfl': 0.5,  # Distribution Focal Loss weight
    'box': 8.5,  # Box loss weight
    'plots': True,  # Enable plotting
    'weight_decay': 0.002,  # Weight decay
    'val': True,  # Enable validation
    'patience': 10  # Early stopping if no improvement for 10 epochs
}

# Train the model
model.train(data='/kaggle/working/data.yaml', **kwargs)

# Check if model checkpoint exists before saving
if model.ckpt:
    # Save the model after training
    model.save('/kaggle/working/trained_model.pt')
else:
    print("Error: Model checkpoint not found. Saving failed.")

# Use Weights & Biases API to download a model artifact
api = wandb.Api(api_key='fb73c6108b6288bb98745c5ae84e62279eb76cb0')
artifact = api.artifact('personalid/YOLOv8/run_6m0x81iv_model:v0', type='model')
artifact_dir = artifact.download('/kaggle/working/yolo-traffic-detectV2')

# Finish the wandb run
wandb.finish()
print("Finished Training")


Tue Jun 25 03:19:44 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Ultralytics YOLOv8.2.42 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
                                                      CUDA:1 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=/kaggle/working/data.yaml, epochs=1, time=None, patience=10, batch=16, imgsz=640, save=True, save_period=1, cache=False, device=[0, 1], workers=8, project=Traffic_Analysis, name=First, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, sa

100%|██████████| 755k/755k [00:00<00:00, 13.9MB/s]


Overriding model.yaml nc=80 with nc=5

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

...

                   all       1167       3895      0.287      0.141     0.0566     0.0198
         auto-rickshaw        371        704      0.178      0.251      0.123     0.0396
                   bus        193        291     0.0371      0.326     0.0581      0.021
                   car        580       2031      0.128     0.0935     0.0433     0.0169
            motorcycle        203        440          1    0.00596     0.0428     0.0158
                 truck        260        429     0.0912     0.0256     0.0154    0.00591
Speed: 0.2ms preprocess, 2.1ms inference, 0.0ms loss, 1.6ms postprocess per image
Results saved to Traffic_Analysis/First


wandb:                                                                                
wandb: 
wandb: Run history:
wandb:                  lr/pg0 ▁
wandb:                  lr/pg1 ▁
wandb:                  lr/pg2 ▁
wandb:        metrics/mAP50(B) ▁
wandb:     metrics/mAP50-95(B) ▁
wandb:    metrics/precision(B) ▁
wandb:       metrics/recall(B) ▁
wandb:            model/GFLOPs ▁
wandb:        model/parameters ▁
wandb: model/speed_PyTorch(ms) ▁
wandb:          train/box_loss ▁
wandb:          train/cls_loss ▁
wandb:          train/dfl_loss ▁
wandb:            val/box_loss ▁
wandb:            val/cls_loss ▁
wandb:            val/dfl_loss ▁
wandb: 
wandb: Run summary:
wandb:                  lr/pg0 0.00024
wandb:                  lr/pg1 0.00024
wandb:                  lr/pg2 0.00024
wandb:        metrics/mAP50(B) 0.0566
wandb:     metrics/mAP50-95(B) 0.01985
wandb:    metrics/precision(B) 0.28688
wandb:       metrics/recall(B) 0.14061
wandb:            model/GFLOPs 8.198
wandb:        model/

Error: Model checkpoint not found. Saving failed.


wandb:   1 of 1 files downloaded.  


Finished Training
